<a href="https://colab.research.google.com/github/cbonnin88/TerraLoop/blob/main/TerraLoop_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import polars as pl
import plotly.express as px
import random
from datetime import datetime, timedelta
import plotly.colors

In [72]:
df_events = pl.read_csv('tl_events_polars.csv')
df_users = pl.read_csv('tl_users_polars.csv')
df_listings = pl.read_csv('tl_listings_polars.csv')

In [73]:
df_events = df_events.with_columns(
    pl.col("user_id").str.strip_chars().str.to_uppercase()
)

df_users = df_users.with_columns(
    pl.col("user_id").str.strip_chars().str.to_uppercase()
)

In [74]:
df_events.head()

user_id,event_type,event_time,platform,session_id,item_category
str,str,str,str,i64,str
"""TL-0429""","""start_listing""","""2026-01-26T14:00:00.000000""","""android""",130687,"""Battery"""
"""TL-0233""","""start_listing""","""2026-02-05T23:00:00.000000""","""web""",331073,"""Small Appliance"""
"""TL-0086""","""search_item""","""2026-02-03T19:00:00.000000""","""ios""",588636,"""Battery"""
"""TL-0288""","""start_listing""","""2026-02-15T07:00:00.000000""","""ios""",694530,"""Battery"""
"""TL-0104""","""start_listing""","""2026-01-21T07:00:00.000000""","""web""",849706,"""Battery"""


In [75]:
df_users.head()

user_id,signup_date,user_tier,region
str,str,str,str
"""TL0000""","""2026-01-22T00:00:00.000000""","""Basic""","""Asia"""
"""TL0001""","""2026-01-13T00:00:00.000000""","""Basic""","""Asia"""
"""TL0002""","""2026-01-29T00:00:00.000000""","""Basic""","""Asia"""
"""TL0003""","""2026-01-09T00:00:00.000000""","""Eco-Ally""","""Asia"""
"""TL0004""","""2026-01-10T00:00:00.000000""","""Basic""","""North America"""


In [76]:
df_listings.head()

listing_id,owner_id,eco_score_impact,status,created_at
str,str,i64,str,str
"""LST-00000""","""TL-0021""",32,"""cancelled""","""2026-01-26T00:00:00.000000"""
"""LST-00001""","""TL-0489""",53,"""completed""","""2026-01-27T00:00:00.000000"""
"""LST-00002""","""TL-0138""",18,"""completed""","""2026-01-06T00:00:00.000000"""
"""LST-00003""","""TL-0277""",100,"""active""","""2026-01-26T00:00:00.000000"""
"""LST-00004""","""TL-0479""",37,"""cancelled""","""2026-01-27T00:00:00.000000"""


In [11]:
print("Listings IDs sample:", df_listings["owner_id"].head(5).to_list())
print("Users IDs sample:", df_users["user_id"].head(5).to_list())

Listings IDs sample: ['TL-0021', 'TL-0489', 'TL-0138', 'TL-0277', 'TL-0479']
Users IDs sample: ['TL0000', 'TL0001', 'TL0002', 'TL0003', 'TL0004']


In [12]:
df_users = df_users.with_columns(pl.col("user_id").str.strip_chars().str.to_uppercase())
df_listings = df_listings.with_columns(pl.col("owner_id").str.strip_chars().str.to_uppercase())

In [13]:
df_users.head()

user_id,signup_date,user_tier,region
str,str,str,str
"""TL0000""","""2026-01-22T00:00:00.000000""","""Basic""","""Asia"""
"""TL0001""","""2026-01-13T00:00:00.000000""","""Basic""","""Asia"""
"""TL0002""","""2026-01-29T00:00:00.000000""","""Basic""","""Asia"""
"""TL0003""","""2026-01-09T00:00:00.000000""","""Eco-Ally""","""Asia"""
"""TL0004""","""2026-01-10T00:00:00.000000""","""Basic""","""North America"""


In [14]:
df_listings.head()

listing_id,owner_id,eco_score_impact,status,created_at
str,str,i64,str,str
"""LST-00000""","""TL-0021""",32,"""cancelled""","""2026-01-26T00:00:00.000000"""
"""LST-00001""","""TL-0489""",53,"""completed""","""2026-01-27T00:00:00.000000"""
"""LST-00002""","""TL-0138""",18,"""completed""","""2026-01-06T00:00:00.000000"""
"""LST-00003""","""TL-0277""",100,"""active""","""2026-01-26T00:00:00.000000"""
"""LST-00004""","""TL-0479""",37,"""cancelled""","""2026-01-27T00:00:00.000000"""


In [19]:
# Forcing listings to only use valid user_ids
existing_ids = df_users['user_id'].to_list()

df_listings = df_listings.with_columns(
    pl.Series('owner_id',[random.choice(existing_ids) for _ in range(len(df_listings))])
)

# **Enriching the listings**

- We want to know the Tier fo the person who created the listing

In [122]:
# Filter df_events for 'start_listing' events and select relevant columns
start_listing_details = df_events.filter(pl.col('event_type') == 'start_listing').select(
    'user_id',
    'event_time',
    'item_category'
)

# Join df_listings with start_listing_details to get item_category
# Assumes a listing's created_at matches an event_time for a start_listing event by the same user
df_listings_with_category = df_listings.join(
    start_listing_details,
    left_on=['owner_id', 'created_at'],
    right_on=['user_id', 'event_time'],
    how='left' # Use left join to keep all listings
)

# Then join with df_users to get user details (user_tier, region etc.)
df_listings_enriched = df_listings_with_category.join(
    df_users,
    left_on='owner_id',
    right_on='user_id',
    how='left'
)

In [21]:
df_listings_enriched.head()

listing_id,owner_id,eco_score_impact,status,created_at,signup_date,user_tier,region
str,str,i64,str,str,str,str,str
"""LST-00000""","""TL0275""",32,"""cancelled""","""2026-01-26T00:00:00.000000""","""2026-01-02T00:00:00.000000""","""Basic""","""Asia"""
"""LST-00001""","""TL0373""",53,"""completed""","""2026-01-27T00:00:00.000000""","""2026-01-08T00:00:00.000000""","""Basic""","""North America"""
"""LST-00002""","""TL0432""",18,"""completed""","""2026-01-06T00:00:00.000000""","""2026-01-27T00:00:00.000000""","""Basic""","""Europe"""
"""LST-00003""","""TL0421""",100,"""active""","""2026-01-26T00:00:00.000000""","""2026-01-24T00:00:00.000000""","""Basic""","""Europe"""
"""LST-00004""","""TL0018""",37,"""cancelled""","""2026-01-27T00:00:00.000000""","""2026-01-06T00:00:00.000000""","""Basic""","""North America"""


# **Calculating Total Eco-Impract by Tier**

In [22]:
impact_summary = (
    df_listings_enriched.group_by('user_tier')
    .agg(
        pl.col('eco_score_impact').sum().alias('total_impact'),
        pl.col('listing_id').count().alias('listing_count')
    )
    .with_columns(
        (pl.col('total_impact') / pl.col('listing_count')).alias('avg_impact_per_listing')
    )
)

display(impact_summary)

user_tier,total_impact,listing_count,avg_impact_per_listing
str,i64,u32,f64
"""Basic""",28337,518,54.704633
"""Eco-Ally""",11782,224,52.598214
"""Premium""",3116,58,53.724138


# **Impact vs Volume**

In [23]:
fig1 = px.bar(
    impact_summary.to_pandas(),
    x='user_tier',
    y='total_impact',
    color='user_tier',
    text_auto='.2s',
    title='Total Eco-Score Impact by User Tier',
    labels={'total_impact':'Total Eco-Score Points','user_tier':'Membership Tier'}
)

fig1.update_layout(showlegend=False)
fig1.show()

# **Active Listing Density by Region**

- Which geographics areas are most engaged with the circular economy ?

In [32]:
region_stats = (
    df_listings_enriched.group_by('region')
    .agg(
        pl.col('listing_id').count().alias('total_listings'),
        pl.col('owner_id').n_unique().alias('unique_sellers')
    )
    .with_columns(
        (pl.col('total_listings') / pl.col('unique_sellers')).alias('listings_per_user'),
        pl.col('total_listings').cast(pl.Float64)
    )
)

display(region_stats)

region,total_listings,unique_sellers,listings_per_user
str,f64,u32,f64
"""North America""",266.0,133,2.0
"""Asia""",244.0,130,1.876923
"""Europe""",290.0,136,2.132353


In [35]:
fig_reg = px.bar(
    region_stats.to_pandas(),
    x='region',
    y='listings_per_user',
    color='total_listings',
    title='Average Listings per Active User by Region',
    labels={'listings_per_user':'Listings Per User','region':'Region'},
    color_continuous_scale='Viridis'
)

fig_reg.update_layout(coloraxis_showscale=False)
fig_reg.show()

# **Eco-Impact by User Tier**
- Tracking how much *'Green Value'* (Eco-Score) each tier generates over time.
- This is our **North Star Metric**

In [39]:
# Preparing the time-series data
# Only convert if 'created_at' is not already a date type
if df_listings_enriched['created_at'].dtype != pl.Date:
    df_listings_enriched = df_listings_enriched.with_columns(
        pl.col('created_at').str.to_datetime("%Y-%m-%dT%H:%M:%S%.f").cast(pl.Date)
    )

display(df_listings_enriched.head())

listing_id,owner_id,eco_score_impact,status,created_at,signup_date,user_tier,region
str,str,i64,str,date,str,str,str
"""LST-00000""","""TL0275""",32,"""cancelled""",2026-01-26,"""2026-01-02T00:00:00.000000""","""Basic""","""Asia"""
"""LST-00001""","""TL0373""",53,"""completed""",2026-01-27,"""2026-01-08T00:00:00.000000""","""Basic""","""North America"""
"""LST-00002""","""TL0432""",18,"""completed""",2026-01-06,"""2026-01-27T00:00:00.000000""","""Basic""","""Europe"""
"""LST-00003""","""TL0421""",100,"""active""",2026-01-26,"""2026-01-24T00:00:00.000000""","""Basic""","""Europe"""
"""LST-00004""","""TL0018""",37,"""cancelled""",2026-01-27,"""2026-01-06T00:00:00.000000""","""Basic""","""North America"""


In [57]:
impact_over_time = (
    df_listings_enriched.group_by(['created_at','user_tier'])
    .agg(pl.col('eco_score_impact').sum().alias('daily_impact'))
    .sort('created_at')
    .with_columns(
        pl.col('daily_impact').cum_sum().over('user_tier').alias('total_impact')
    )
)

display(impact_over_time.head())

created_at,user_tier,daily_impact,total_impact
date,str,i64,i64
2026-01-01,"""Premium""",265,265
2026-01-01,"""Eco-Ally""",582,582
2026-01-01,"""Basic""",696,696
2026-01-02,"""Eco-Ally""",118,700
2026-01-02,"""Basic""",728,1424


In [58]:
fig_impact = px.line(
    impact_over_time.to_pandas(),
    x='created_at',
    y='total_impact',
    color='user_tier',
    title='Total Eco-Impact Growth over Time'
)

fig_impact.show()

In [49]:
impact_over_time = impact_over_time.with_columns(
    pl.col('daily_impact')
    .rolling_mean(window_size=7)
    .over('user_tier')
    .alias('rolling_avg_impact')
)

display(impact_over_time)

created_at,user_tier,daily_impact,cumulative_impact,rolling_avg_impact
date,str,i64,i64,f64
2026-01-01,"""Basic""",696,696,null
2026-01-01,"""Premium""",265,265,null
2026-01-01,"""Eco-Ally""",582,582,null
2026-01-02,"""Eco-Ally""",118,700,null
2026-01-02,"""Basic""",728,1424,null
…,…,…,…,…
2026-02-09,"""Eco-Ally""",325,11622,335.857143
2026-02-09,"""Basic""",510,27413,615.285714
2026-02-10,"""Eco-Ally""",160,11782,292.571429


In [55]:
fig_rolling_avg = px.line(
    impact_over_time.to_pandas(),
    x='created_at',
    y='rolling_avg_impact',
    color='user_tier',
    title='7-Day Rolling Average Eco-Impact by User Tier',
    labels={'created_at':'Date','rolling_avg_impact':'7-Day Avg Impact','user_tier':'Tier'},
    template='plotly_white'
)

fig_rolling_avg.update_traces(line=dict(width=3))
fig_rolling_avg.update_layout(
    hovermode='x unified',
    yaxis_title='Eco-Score Impact (Rolling Average)',
    xaxis_title='Timeline'
)
fig_rolling_avg.show()

# **The Conversion Funnel**
- How many users make if from *session_start* to c*omplete_listing* ?

In [79]:
# Force the events to use ONLY IDs that exist in df_users
valid_ids = df_users["user_id"].unique().to_list()

df_events = df_events.with_columns(
    pl.Series("user_id", [random.choice(valid_ids) for _ in range(len(df_events))])
)

In [80]:
# Enrich events with user data
df_enriched = df_events.join(df_users, on="user_id", how="left")

In [82]:
# Aggregate unique users per stage and tier
funnel_data = (
    df_enriched.group_by(['event_type','user_tier'])
    .agg(pl.col("user_id").n_unique().alias('unique_users'))
    .sort('unique_users',descending=True)
)

display(funnel_data)

event_type,user_tier,unique_users
str,str,u32
"""session_start""","""Basic""",324
"""search_item""","""Basic""",324
"""start_listing""","""Basic""",324
"""complete_listing""","""Basic""",311
"""session_start""","""Eco-Ally""",139
…,…,…
"""complete_listing""","""Eco-Ally""",134
"""session_start""","""Premium""",37
"""start_listing""","""Premium""",37


In [86]:
fig_funnel = px.funnel(
    funnel_data.to_pandas(),
    x='unique_users',
    y='event_type',
    color='user_tier',
    title='TerraLoop: Listing Funnel by Tier',
    labels={'unique_users':'Users','user_tier':'Tier'}
)

fig_funnel.show()

# **Stickiness Ration (DAU/MAU)**
- How many users come back daily vs. monthly ?

In [88]:
# Convert timestamp to a date
# Only convert if 'event_time' is not already a date type
if df_enriched['event_time'].dtype != pl.Date:
    df_enriched = df_enriched.with_columns(pl.col('event_time').str.to_datetime("%Y-%m-%dT%H:%M:%S%.f").cast(pl.Date).alias('date'))
else:
    df_enriched = df_enriched.with_columns(pl.col('event_time').alias('date'))

In [91]:
dau = df_enriched.group_by("date").agg(pl.col('user_id').n_unique().alias('dau'))
mau = df_enriched.select(pl.col("user_id").n_unique()).item() # Total unique users in the month

stickiness = (dau.select(pl.col('dau').mean()).item() / mau) * 100
print(f'App Stickiness: {stickiness:.1f}%')

App Stickiness: 52.9%


# **Event Category Affinity**
- This tells us if Pro users are more interested in Industrial categories while Basic users stick to Clothing/Electronics

In [101]:
# 1. Clean the keys for the join
df_events = df_events.with_columns(pl.col("user_id").str.strip_chars().str.to_uppercase())
df_users = df_users.with_columns(pl.col("user_id").str.strip_chars().str.to_uppercase())

# 2. Join events with users
# This ensures every event row now has 'item_category' AND 'user_tier'
df_enriched = df_events.join(df_users, on="user_id", how="left")

# 3. Verify the columns are there
print("Columns in enriched events:", df_enriched.columns)

Columns in enriched events: ['user_id', 'event_type', 'event_time', 'platform', 'session_id', 'item_category', 'signup_date', 'user_tier', 'region']


In [103]:
# Aggregate unique users per category and tier
affinity_data = (
    df_enriched.group_by(["item_category", "user_tier"])
    .agg(pl.col("user_id").n_unique().alias("unique_users"))
    .sort("unique_users", descending=True)
)

display(affinity_data.head())

item_category,user_tier,unique_users
str,str,u32
"""Phone""","""Basic""",324
"""Laptop""","""Basic""",324
"""Battery""","""Basic""",324
"""Small Appliance""","""Basic""",324
"""Laptop""","""Eco-Ally""",139


In [105]:
fig_affinity = px.parallel_categories(
    affinity_data.to_pandas(),
    dimensions=['user_tier', 'item_category'],
    title="<b>User Interest: Tier to Category Mapping</b>",
    color="unique_users",
    color_continuous_scale="Tealgrn"
)
fig_affinity.show()

# **Category Conversion "Stickers"**

- Which category has the **highest completion rate.**

In [107]:
# 1. Calculate Start vs Complete per category
cat_conversion = (
    df_enriched.group_by(["item_category"])
    .agg([
        pl.col("event_type").filter(pl.col("event_type") == "start_listing").count().alias("starts"),
        pl.col("event_type").filter(pl.col("event_type") == "complete_listing").count().alias("completes")
    ])
    .with_columns(
        (pl.col("completes") / pl.col("starts")).alias("conversion_rate")
    )
    .sort("conversion_rate", descending=True)
)

display(cat_conversion)

item_category,starts,completes,conversion_rate
str,u32,u32,f64
"""Small Appliance""",1010,395,0.391089
"""Battery""",1043,383,0.36721
"""Phone""",981,353,0.359837
"""Laptop""",1017,352,0.346116


In [108]:
fig_cat_conv = px.bar(
    cat_conversion.to_pandas(),
    x="conversion_rate",
    y="item_category",
    orientation='h',
    title="<b>Listing Completion Rate by Category</b>",
    color="conversion_rate",
    text_auto='.1%', # Shows the percentage on the bar
    color_continuous_scale="RdYlGn"
)
fig_cat_conv.show()

# **Category Dominance (Sunburst Chart)**

In [125]:
sunburst_data = (
    df_listings_enriched.group_by(["region", "item_category", "user_tier"])
    .agg(pl.col("listing_id").count().alias("count"))
    .filter(pl.col('item_category').is_not_null()) # Filter out rows where item_category is null
)

display(sunburst_data)

region,item_category,user_tier,count
str,str,str,u32
"""Europe""","""Laptop""","""Eco-Ally""",2
"""Asia""","""Laptop""","""Basic""",2
"""North America""","""Phone""","""Basic""",2
"""Asia""","""Laptop""","""Eco-Ally""",1
"""North America""","""Small Appliance""","""Eco-Ally""",1


In [126]:
fig_sun = px.sunburst(
    sunburst_data.to_pandas(),
    path=['region', 'item_category', 'user_tier'],
    values='count',
    title="<b>TerraLoop Ecosystem: Where is the Waste Coming From?</b>",
    color='count',
    color_continuous_scale='RdYlGn' # Red to Green for Eco-Impact
)
fig_sun.show()

# **Listing Lead Time**
- How many days pass between when a user joins and when they post their first listing?

In [118]:
activation_df = (
    df_listings_enriched.with_columns([
        pl.col("created_at").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f").cast(pl.Date),
        pl.col("signup_date").str.to_datetime("%Y-%m-%dT%H:%M:%S%.f").cast(pl.Date)
    ])
    .with_columns(
        (pl.col("created_at") - pl.col("signup_date")).dt.total_days().alias("days_to_list")
    )
)

display(activation_df.head())

listing_id,owner_id,eco_score_impact,status,created_at,signup_date,user_tier,region,days_to_list
str,str,i64,str,date,date,str,str,i64
"""LST-00000""","""TL0200""",32,"""cancelled""",2026-01-26,2026-01-31,"""Basic""","""North America""",-5
"""LST-00001""","""TL0337""",53,"""completed""",2026-01-27,2026-01-20,"""Basic""","""Europe""",7
"""LST-00002""","""TL0129""",18,"""completed""",2026-01-06,2026-01-06,"""Eco-Ally""","""North America""",0
"""LST-00003""","""TL0197""",100,"""active""",2026-01-26,2026-01-18,"""Eco-Ally""","""Asia""",8
"""LST-00004""","""TL0357""",37,"""cancelled""",2026-01-27,2026-01-31,"""Basic""","""Asia""",-4


In [119]:
fig_dist = px.histogram(
    activation_df.to_pandas(),
    x="days_to_list",
    color="user_tier",
    marginal="box", # Adds a box plot on top
    title="<b>Days to First Listing: User Activation Speed</b>",
    labels={"days_to_list": "Days from Signup to Listing"}
)
fig_dist.show()